In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
from scipy import stats

In [ ]:
ds = xr.open_dataset("./data/wind_europe_2015_2019.nc")

In [ ]:
wind_speed = np.sqrt(ds['u100']**2 + ds['v100']**2)

In [ ]:
mean_wind_speed = wind_speed.stack(cell=["latitude", "longitude"])
mean_wind_speed = mean_wind_speed.groupby("cell").mean(...).unstack("cell")
mean_wind_speed = mean_wind_speed.rename({ "cell_level_0": "latitude", "cell_level_1": "longitude" })

In [ ]:
fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
# ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
plot1 = mean_wind_speed.plot(cmap=plt.cm.YlGnBu, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink':0.4})

In [ ]:
swept_area = 38000
power = 12
def reduce_func(wind):
    power_in = 0.5 * 1.247 * swept_area * wind**3 / 10**6 
    return xr.where(np.logical_and(wind > 3, wind < 25), np.minimum(1, 16/27 * power_in / power), 0).mean()

min_wind_speed = wind_speed.stack(cell=["latitude", "longitude"])
min_wind_speed = min_wind_speed.groupby("cell").reduce(reduce_func, ...).unstack("cell")
min_wind_speed = min_wind_speed.rename({ "cell_level_0": "latitude", "cell_level_1": "longitude" })

In [ ]:
fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
#ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
plot1 = min_wind_speed.plot(cmap=plt.cm.YlGnBu, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink':0.4})

In [ ]:
x, y = np.meshgrid(ds['longitude'].values, ds['latitude'].values)
u = ds['u100'].mean(['time']).values
v = ds['v100'].mean(['time']).values

In [ ]:
fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
# ax.set_extent([-12, 32, 35, 60], crs=ccrs.PlateCarree())
ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
ax.quiver(x, y, u, v, transform=ccrs.PlateCarree())
plt.show()

In [ ]:
# inverser quantile
# groupe de vent
# vent minimale sur la zone ??? Production ? 
# carte vecteurs